In [51]:
import numpy as np # библиотека для работы с чиселками
import os
import pandas as pd # data processing, работа с CSV файлами
import matplotlib.pyplot as plt # для графики
import seaborn as sns # аналогично
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import Subset
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torchmetrics.classification import BinaryJaccardIndex, BinaryF1Score, BinaryPrecision, BinaryRecall
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import DataLoader, TensorDataset
import segmentation_models_pytorch as smp

Откроем описание датасета в формате CSV и посмотрим первые 5 строчек

In [2]:
dataset = pd.read_csv('Coffee_bean_dataset\\Coffee_Bean.csv', encoding='ISO-8859-1')
dataset.head(5)

,class index,filepaths,labels,data set
0,0,train/Dark/dark (1).png,Dark,train
1,0,train/Dark/dark (10).png,Dark,train
2,0,train/Dark/dark (100).png,Dark,train
3,0,train/Dark/dark (101).png,Dark,train
4,0,train/Dark/dark (102).png,Dark,train


In [29]:
def load_data(image_dir, label_dir, transform):
    image_filenames = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

    images = []
    labels = []
    
    for filename in image_filenames:
        # Загружаем изображение
        img_path = os.path.join(image_dir, filename)
        img = Image.open(img_path).convert("RGB")
        img_np = np.array(img)
        #img_tensor = transform(img)
        #images.append(img_np)

        # Загружаем соответствующую маску
        label_path = os.path.join(label_dir, filename.replace(".jpg", ".png"))
        label = Image.open(label_path).convert("L")
        label = label.resize((224, 224), Image.NEAREST)
        label_np = np.array(label)
        
        transformed = transform(image=img_np, mask=label_np)

        labels.append(transformed['mask'].float())
        images.append(transformed['image'].float())

    return images, labels

In [100]:
# Трансформации
transform = A.Compose([
    ToTensorV2()
])

In [101]:
image_dir_train = "C:\\Users\\Aila\\Уроки\\Multimedia_2nd_semester\\Coffee_bean_dataset\\segmentation_train\\images"
label_dir_train = "C:\\Users\\Aila\\Уроки\\Multimedia_2nd_semester\\Coffee_bean_dataset\\segmentation_train\\labels"
image_dir_test = "C:\\Users\\Aila\\Уроки\\Multimedia_2nd_semester\\Coffee_bean_dataset\\segmentation_test\\images"
label_dir_test = "C:\\Users\\Aila\\Уроки\\Multimedia_2nd_semester\\Coffee_bean_dataset\\segmentation_test\\labels"

X_train, y_train = load_data(image_dir_train, label_dir_train, transform)
X_test, y_test = load_data(image_dir_test, label_dir_test, transform)

In [ ]:
# Преобразуем данные в TensorDataset
dataset = TensorDataset(torch.stack(X_train), torch.stack(y_train))
dataset_test = TensorDataset(torch.stack(X_test), torch.stack(y_test))


Теперь обучим модели Unet и DeepLabV3.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Обучим модели и посмотрим на результаты:

In [115]:
def train(model, model_name, dataset, num_of_epochs=5):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    
    # Создаем DataLoader с батчами
    train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

    # Обучение
    for epoch in range(num_of_epochs):
        model.train()
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            if y_batch.ndim == 4 and y_batch.shape[1] == 1:
                y_batch = y_batch.squeeze(1)  # from (B, 1, H, W) → (B, H, W)
            y_batch = (y_batch > 127).long()
            optimizer.zero_grad()
            outputs = model(X_batch)
            #print(1)
            loss = criterion(outputs.squeeze(1), y_batch.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"{model_name}: Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

def eval(model, model_name, test_dataset):
    iou_metric = BinaryJaccardIndex(ignore_index=255).to(device)
    dice_metric = BinaryF1Score(ignore_index=255).to(device)
    precision_metric = BinaryPrecision(ignore_index=255).to(device)
    recall_metric = BinaryRecall(ignore_index=255).to(device)
    
    iou_metric.reset()
    dice_metric.reset()
    precision_metric.reset()
    recall_metric.reset()

    # Создаем DataLoader с батчами
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

    model.eval()
    #y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            if y_batch.ndim == 4 and y_batch.shape[1] == 1:
                y_batch = y_batch.squeeze(1)  # from (B, 1, H, W) → (B, H, W)
            outputs = model(X_batch)
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.3).squeeze(1).long()
            y_batch = (y_batch > 127).long()
            #preds = torch.sigmoid(outputs)  # Вероятности для класса объект
            #preds = (preds > 0.5).squeeze(1).long()  # Преобразуем в бинарные метки (0 или 1)
            #plt.imshow(preds[0].numpy(), cmap='gray')
            #print(1)
            #plt.imshow(preds[0].squeeze(0).numpy(), cmap='gray')
            #print(2)
            #plt.axis('off')
            #plt.show()
            # Обновляем метрики
            iou_metric.update(preds, y_batch)
            dice_metric.update(preds, y_batch)
            precision_metric.update(preds, y_batch)
            recall_metric.update(preds, y_batch)

    print(f"\nEvaluation Metrics for {model_name}:")
    print("mIoU:", iou_metric.compute().item())
    print("Dice:", dice_metric.compute().item())
    print("Precision:", precision_metric.compute().item())
    print("Recall:", recall_metric.compute().item())

In [ ]:

unet = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
).to(device)
train(unet, "Unet", dataset=dataset, num_of_epochs=1)

eval(unet, "Unet", test_dataset=dataset_test)


Evaluation Metrics for Unet:
mIoU: 0.9638083577156067
Dice: 0.9815707206726074
Precision: 0.9657078385353088
Recall: 0.9979634284973145


In [98]:
deeplab = smp.DeepLabV3(
    encoder_name="resnet34",     
    encoder_weights="imagenet",
    in_channels=3,             
    classes=1                  
).to(device)

train(deeplab, "Deeplab", dataset=dataset, num_of_epochs=3)
eval(deeplab, "Deeplab", test_dataset=dataset_test)

Deeplab: Epoch 1, Loss: 0.1118
Deeplab: Epoch 2, Loss: 0.0424
Deeplab: Epoch 3, Loss: 0.0296

Evaluation Metrics for Deeplab:
mIoU: 0.9625186324119568
Dice: 0.9809014201164246
Precision: 0.9641741514205933
Recall: 0.9982192516326904


In [99]:
torch.save(unet, "unet_full.pth")
torch.save(deeplab, "deeplabv3_full.pth")

Вывод:
1. Метрики для Unet:
Модель Unet показала высокие значения по всем ключевым метрикам, что свидетельствует о стабильной и точной сегментации с минимальными пропущенными пикселями объектов. Высокий Recall говорит о том, что модель почти не упускает истинные положительные объекты, а высокие Precision и Dice подтверждают, что ложноположительных предсказаний почти нет.
2. Метрики для DeepLabv3:
DeepLabv3 также показал почти идентичные показатели. Это указывает на сопоставимую производительность с Unet, но немного ниже precision, что может говорить о чуть большей склонности к ложноположительным предсказаниям. Однако разница крайне мала — обе модели демонстрируют высокую обобщающую способность и уверенно решают задачу.

Обе архитектуры успешно справляются с поставленной задачей и пригодны для практического применения без необходимости глубокой дополнительной настройки. Модели не переобучились, предсказания уверенные и согласованные по метрикам.

# Улучшение бейзлайна

Для улучшения бейзлайна модели в задачи семантической сегментации предлагаю следующие решения:

Провести аугментацию тренировочных данных: использовать нормализацию, повороты и изменить яркость и контраст изображений.

Аугментация тренировочного датасета:

In [103]:

aug_transform = A.Compose([
    A.Rotate(limit=15, p=1.0),  # случайный поворот до 15 градусов
    A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=70, val_shift_limit=40, p=1.0),  # изменение оттенка, насыщенности, яркости
    A.Normalize(),
    ToTensorV2()
])

X_train_new, y_train_new = load_data(image_dir_train, label_dir_train, aug_transform)
X_test_new, y_test_new = load_data(image_dir_test, label_dir_test, aug_transform)

dataset_new = TensorDataset(torch.stack(X_train_new), torch.stack(y_train_new))
dataset_test_new = TensorDataset(torch.stack(X_test_new), torch.stack(y_test_new))

In [104]:
unet_new = smp.Unet(
    encoder_name="resnet34",     
    encoder_weights="imagenet",
    in_channels=3,             
    classes=1                  
).to(device)

train(unet_new, "Unet", dataset=dataset_new, num_of_epochs=3)
eval(unet_new, "Unet", test_dataset=dataset_test_new)

Unet: Epoch 1, Loss: 0.2898
Unet: Epoch 2, Loss: 0.1006
Unet: Epoch 3, Loss: 0.0615

Evaluation Metrics for Unet:
mIoU: 0.9717341065406799
Dice: 0.9856644868850708
Precision: 0.9758588671684265
Recall: 0.9956691265106201


In [105]:
deeplab_new = smp.DeepLabV3(
    encoder_name="resnet34",     
    encoder_weights="imagenet",
    in_channels=3,             
    classes=1                  
).to(device)

train(deeplab_new, "Deeplab", dataset=dataset_new, num_of_epochs=3)
eval(deeplab_new, "Deeplab", test_dataset=dataset_test_new)

Deeplab: Epoch 1, Loss: 0.1342
Deeplab: Epoch 2, Loss: 0.0580
Deeplab: Epoch 3, Loss: 0.0390

Evaluation Metrics for Deeplab:
mIoU: 0.9629967212677002
Dice: 0.9811496138572693
Precision: 0.9659270644187927
Recall: 0.996859610080719


In [106]:
torch.save(unet_new, "unet_new_full.pth")
torch.save(deeplab_new, "deeplabv3_new_full.pth")

Вывод:

Unet после улучшения демонстрирует ещё более высокое качество сегментации. Рост всех метрик — особенно Dice (вырос с 0.98 до 0.99) и mIoU (вырос с 0.96 до 0.97) — говорит о том, что модель стала лучше захватывать границы объектов, снижая количество как пропущенных, так и ошибочно предсказанных пикселей.

DeepLabv3 по-прежнему показывает высокое и стабильное качество, остаётся сильной, особенно по recall (0.99), но немного уступает улучшенному Unet. Это говорит о том, что без дополнительной настройки DeepLabv3 сохраняет достойные результаты, но менее чувствителен к мелким улучшениям по сравнению с Unet.

Таким образом, улучшения в виде аугментаций, подбора гиперпараметров и архитектурных изменений оказали положительное влияние на обе модели.
После доработки Unet стал заметно лучше, выйдя вперёд по всем метрикам. DeepLabv3 по-прежнему силён, особенно в задачах с разными масштабами объектов, но Unet после улучшений — более эффективное решение для данной задачи.

### Имплементация алгоритма

In [118]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50

In [109]:
class DoubleConv(nn.Module):
    """(Conv => BN => ReLU) * 2"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.down = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.down(x)

class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        # Pad x1 to match x2 if needed
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, bilinear=True):
        super().__init__()

        self.in_conv = DoubleConv(in_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)

        self.up1 = Up(1024 + 512, 512, bilinear)
        self.up2 = Up(512 + 256, 256, bilinear)
        self.up3 = Up(256 + 128, 128, bilinear)
        self.up4 = Up(128 + 64, 64, bilinear)

        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.in_conv(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)

        return self.out_conv(x)


In [119]:
class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels, rates=[6, 12, 18]):
        super(ASPP, self).__init__()
        self.blocks = nn.ModuleList()
        self.blocks.append(nn.Sequential(  # 1x1 conv
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        ))
        for rate in rates:
            self.blocks.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=rate, dilation=rate, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            ))
        self.global_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
        self.project = nn.Sequential(
            nn.Conv2d(out_channels * (len(rates) + 2), out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        size = x.shape[2:]
        res = [block(x) for block in self.blocks]
        gp = self.global_pool(x)
        gp = F.interpolate(gp, size=size, mode='bilinear', align_corners=False)
        res.append(gp)
        x = torch.cat(res, dim=1)
        return self.project(x)

class DeepLabV3(nn.Module):
    def __init__(self, num_classes=1, backbone='resnet50', pretrained=True):
        super(DeepLabV3, self).__init__()
        resnet = resnet50(pretrained=pretrained)

        self.backbone = nn.Sequential(
            resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool,
            resnet.layer1, resnet.layer2, resnet.layer3, resnet.layer4  # output stride 32
        )
        self.aspp = ASPP(in_channels=2048, out_channels=256)
        self.classifier = nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, num_classes, 1)
        )

    def forward(self, x):
        input_size = x.shape[2:]
        features = self.backbone(x)
        x = self.aspp(features)
        x = self.classifier(x)
        return F.interpolate(x, size=input_size, mode='bilinear', align_corners=False)


Посмотрим работу имплементированного алгоритма на обычном датасете:

In [116]:
my_unet = UNet(in_channels=3, out_channels=1).to(device)
train(my_unet, "Unet", dataset=dataset, num_of_epochs=3)
eval(my_unet, "Unet", test_dataset=dataset_test)

Unet: Epoch 1, Loss: 0.2447
Unet: Epoch 2, Loss: 0.1845
Unet: Epoch 3, Loss: 0.1563

Evaluation Metrics for Unet:
mIoU: 0.9643996953964233
Dice: 0.9818772673606873
Precision: 0.9661650061607361
Recall: 0.998108983039856


In [120]:
my_deeplabv3 = DeepLabV3(num_classes=1).to(device)
train(my_deeplabv3, "Deeplab", dataset=dataset, num_of_epochs=3)
eval(my_deeplabv3 , "Deeplab", test_dataset=dataset_test)

c:\Users\Aila\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Aila\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Aila/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:16<00:00, 6.40MB/s]


Deeplab: Epoch 1, Loss: 0.1742
Deeplab: Epoch 2, Loss: 0.0948
Deeplab: Epoch 3, Loss: 0.0759

Evaluation Metrics for Deeplab:
mIoU: 0.8728695511817932
Dice: 0.9321199655532837
Precision: 0.8731996417045593
Recall: 0.9995671510696411


А теперь проверим на улучшенном бейзлайне:

In [117]:
my_unet_new = UNet(in_channels=3, out_channels=1).to(device)
train(my_unet_new, "Unet", dataset=dataset_new, num_of_epochs=3)
eval(my_unet_new, "Unet", test_dataset=dataset_test_new)

Unet: Epoch 1, Loss: 0.2978
Unet: Epoch 2, Loss: 0.2246
Unet: Epoch 3, Loss: 0.1888

Evaluation Metrics for Unet:
mIoU: 0.9507742524147034
Dice: 0.9747660160064697
Precision: 0.9530373215675354
Recall: 0.997508704662323


In [121]:
my_deeplabv3_new = DeepLabV3(num_classes=1).to(device)
train(my_deeplabv3_new, "Deeplab", dataset=dataset_new, num_of_epochs=3)
eval(my_deeplabv3_new , "Deeplab", test_dataset=dataset_test_new)

c:\Users\Aila\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Aila\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Deeplab: Epoch 1, Loss: 0.2133
Deeplab: Epoch 2, Loss: 0.1190
Deeplab: Epoch 3, Loss: 0.0934

Evaluation Metrics for Deeplab:
mIoU: 0.8866561055183411
Dice: 0.9399234056472778
Precision: 0.8876857757568359
Recall: 0.9986934661865234


##### Сравнение собственной реализации U-Net и DeepLabV3 до улучшения бейзлайна:
До улучшения бейзлайна U-Net показывает уверенно высокие результаты по всем метрикам: mIoU — 0.964, Dice — 0.981, Precision — 0.966, Recall — 0.998. Это говорит о высокой согласованности предсказаний модели с истинной разметкой и хорошем обобщении. В то же время DeepLabV3 заметно отстаёт: хотя Recall у него также близок к 1.0 (что означает, что модель почти не пропускает объекты), Precision — всего 0.873, что указывает на большое количество ложноположительных срабатываний. mIoU — 0.873 и Dice — 0.932 подтверждают, что качество сегментации хуже, чем у U-Net.

##### Сравнение собственной реализации U-Net и DeepLabV3 после улучшения бейзлайна:
После доработки бейзлайна качество обеих моделей выросло, но в разной степени. U-Net по-прежнему лидирует: mIoU = 0.950, Dice = 0.975, Precision = 0.953 и Recall = 0.998. Это говорит о сбалансированном и точном предсказании всех классов. DeepLabV3 также немного улучшился, но остаётся менее точным: Dice — 0.940, Precision — 0.887, mIoU — 0.887. Основная проблема по-прежнему заключается в сравнительно низкой точности (Precision), несмотря на высокий Recall, что может свидетельствовать о переизбыточных предсказаниях.

### Вывод:
Собственная реализация U-Net демонстрирует наилучшие метрики как до, так и после улучшения бейзлайна, уверенно опережая DeepLabV3. Последний улучшился, но не догнал U-Net по качеству. Это может быть связано с тем, что U-Net более эффективно использует пространственную информацию в задачах бинарной сегментации, в то время как DeepLabV3 требует более тонкой настройки или сложной архитектурной адаптации.